In [ ]:
import pandas as pd
import numpy as ny
import whois
import json
import requests
import datetime
import urllib.request
import regex
from tldextract import extract
import ssl
import socket
import favicon
import sys, re
import xmltodict
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def url_having_ip(url):
#using regular function
    symbol = regex.findall(r'(http((s)?)://)((((\d)+).)*)((\w)+)(/((\w)+))?',url)
    if(len(symbol)!=0):
        having_ip = 1 #phishing

    else:
        having_ip = -1 #legitimate
    return(having_ip)
    return 0


In [ ]:
def url_length(url):
    length=len(url)
    if(length<54):
        return -1
    elif(54<=length<=75):
        return 0
    else:
        return 1

In [ ]:
def url_short(url):
    try:
        r = requests.get('http://techtv.mit.edu/videos/1585-music-session-02/download.source')
        for i in r.history:
            print(i.url)
            return 0
    except Exception as e:
            return 1

In [ ]:
def having_at_symbol(url):
    symbol=regex.findall(r'@',url)
    if(len(symbol)==0):
        return -1
    else:
        return 1

In [ ]:
def prefix_suffix(url):
    subDomain, domain, suffix = extract(url)
    if(domain.count('-')):
        return 1
    else:
        return -1


In [ ]:
def sub_domain(url):
    subDomain, domain, suffix = extract(url)
    if(subDomain.count('.')==0):
        return -1
    elif(subDomain.count('.')==1):
        return 0
    else:
        return 1

In [ ]:
def SSLfinal_State(url):
    try:
#check wheather contains https
        if(regex.search('^https',url)):
            usehttps = 1
        else:
            usehttps = 0
#getting the certificate issuer to later compare with trusted issuer
        #getting host name
        subDomain, domain, suffix = extract(url)
        host_name = domain + "." + suffix
        context = ssl.create_default_context()
        sct = context.wrap_socket(socket.socket(), server_hostname = host_name)
        sct.connect((host_name, 443))
        certificate = sct.getpeercert()
        issuer = dict(x[0] for x in certificate['issuer'])
        certificate_Auth = str(issuer['commonName'])
        certificate_Auth = certificate_Auth.split()
        if(certificate_Auth[0] == "Network" or certificate_Auth == "Deutsche"):
            certificate_Auth = certificate_Auth[0] + " " + certificate_Auth[1]
        else:
            certificate_Auth = certificate_Auth[0]
        trusted_Auth = ['Comodo','Symantec','GoDaddy','GlobalSign','DigiCert','StartCom','Entrust','Verizon','Trustwave','Unizeto','Buypass','QuoVadis','Deutsche Telekom','Network Solutions','SwissSign','IdenTrust','Secom','TWCA','GeoTrust','Thawte','Doster','VeriSign']
#getting age of certificate
        startingDate = str(certificate['notBefore'])
        endingDate = str(certificate['notAfter'])
        startingYear = int(startingDate.split()[3])
        endingYear = int(endingDate.split()[3])
        Age_of_certificate = endingYear-startingYear

#checking final conditions
        if((usehttps==1) and (certificate_Auth in trusted_Auth) and (Age_of_certificate>=1) ):
            return -1 #legitimate
        elif((usehttps==1) and (certificate_Auth not in trusted_Auth)):
            return 0 #suspicious
        else:
            return 1 #phishing

    except Exception as e:

        return 1

In [ ]:
def domain_registration(url):
    try:
        w = whois.whois(url)
        updated = w.updated_date
        exp = w.expiration_date
        length = (exp[0]-updated[0]).days
        if(length<=365):
            return 1
        else:
            return -1
    except:
        return 0

In [ ]:
def https_token(url):
    subDomain, domain, suffix = extract(url)
    host =subDomain +'.' + domain + '.' + suffix
    if(host.count('https')): #attacker can trick by putting https in domain part
        return 1
    else:
        return -1

In [ ]:
def request_url(url):
    try:
        subDomain, domain, suffix = extract(url)
        websiteDomain = domain

        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')
        imgs = soup.findAll('img', src=True)
        total = len(imgs)

        linked_to_same = 0
        avg =0
        for image in imgs:
            subDomain, domain, suffix = extract(image['src'])
            imageDomain = domain
            if(websiteDomain==imageDomain or imageDomain==''):
                linked_to_same = linked_to_same + 1
        vids = soup.findAll('video', src=True)
        total = total + len(vids)

        for video in vids:
            subDomain, domain, suffix = extract(video['src'])
            vidDomain = domain
            if(websiteDomain==vidDomain or vidDomain==''):
                linked_to_same = linked_to_same + 1
        linked_outside = total-linked_to_same
        if(total!=0):
            avg = linked_outside/total

        if(avg<0.22):
            return -1
        elif(0.22<=avg<=0.61):
            return 0
        else:
            return 1
    except:
        return 0

In [ ]:
def url_of_anchor(url):
    try:
        subDomain, domain, suffix = extract(url)
        websiteDomain = domain

        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')
        anchors = soup.findAll('a', href=True)
        total = len(anchors)
        linked_to_same = 0
        avg = 0
        for anchor in anchors:
            subDomain, domain, suffix = extract(anchor['href'])
            anchorDomain = domain
            if(websiteDomain==anchorDomain or anchorDomain==''):
                linked_to_same = linked_to_same + 1
        linked_outside = total-linked_to_same
        if(total!=0):
            avg = linked_outside/total

        if(avg<0.31):
            return -1
        elif(0.31<=avg<=0.67):
            return 0
        else:
            return 1
    except:
        return 0

In [ ]:
def Links_in_tags(url):
    try:
        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')

        no_of_meta =0
        no_of_link =0
        no_of_script =0
        anchors=0
        avg =0
        for meta in soup.find_all('meta'):
            no_of_meta = no_of_meta+1
        for link in soup.find_all('link'):
            no_of_link = no_of_link +1
        for script in soup.find_all('script'):
            no_of_script = no_of_script+1
        for anchor in soup.find_all('a'):
            anchors = anchors+1
        total = no_of_meta + no_of_link + no_of_script+anchors
        tags = no_of_meta + no_of_link + no_of_script
        if(total!=0):
            avg = tags/total

        if(avg<0.25):
            return -1
        elif(0.25<=avg<=0.81):
            return 0
        else:
            return 1
    except:
        return 0

In [ ]:
def email_submit(url):
    try:
        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')
        if(soup.find('mailto:')):
            return 1
        else:
            return -1
    except:
        return 0

In [ ]:
def age_of_domain(url):
    try:
        w = whois.whois(url)
        start_date = w.creation_date
        current_date = datetime.datetime.now()
        age =(current_date-start_date[0]).days
        if(age>=180):
            return -1
        else:
            return 1
    except Exception as e:
        print(e)
        return 0
def statistical(url):
    return 0

In [ ]:
def fav(url):
    try:
        icons = favicon.get(url)
        if len(icons)>0:
            return 0
        else:
            return 1
    except Exception as e:
        return 1

def sfh(url):
    return 0

In [ ]:
def web_traffic(url):
        try:
            xml = urllib.request.urlopen('http://data.alexa.com/data?cli=10&dat=s&url={}'.format(url)).read()
            result= xmltodict.parse(xml)
            data = json.dumps(result).replace("@","")
            data_tojson = json.loads(data)

            url = data_tojson["ALEXA"]["SD"][1]["POPULARITY"]["URL"]
            rank= data_tojson["ALEXA"]["SD"][1]["POPULARITY"]["TEXT"]
            Ranki = int(rank)
            if Ranki>=10000:
                return 1
            elif Ranki<=10000:
                return -1
            else:
                return 0
        except KeyError:
            return 1
def abnormal_url(url):
    return 0
def redirect(url):
    return 0
def links_pointing(url):
    return 0
def page_rank(url):
    try:
        xml = urllib.request.urlopen('http://data.alexa.com/data?cli=10&dat=s&url={}'.format(url)).read()
        result= xmltodict.parse(xml)
        data = json.dumps(result).replace("@","")
        data_tojson = json.loads(data)

        url = data_tojson["ALEXA"]["SD"][1]["POPULARITY"]["URL"]
        rank= data_tojson["ALEXA"]["SD"][1]["POPULARITY"]["TEXT"]
        Ranki = int(rank)
        if Ranki>=10000:
            return 1
        elif Ranki<=10000:
            return -1
        else:
            return 0
    except KeyError:
        return 1





In [ ]:
def main(url):
    check = [[url_having_ip(url),url_length(url),url_short(url),having_at_symbol(url),
              prefix_suffix(url),sub_domain(url),SSLfinal_State(url),
              domain_registration(url),fav(url),https_token(url),request_url(url),
              url_of_anchor(url),Links_in_tags(url),sfh(url),email_submit(url),abnormal_url(url),
              redirect(url),age_of_domain(url),web_traffic(url),page_rank(url),
              links_pointing(url),statistical(url)]]
    print(check)
    return check


In [ ]:
url = input()
checko = main(url)
print(checko)

https://www.google.com/
http://techtv.mit.edu/videos/1585-music-session-02/download.source
timed out
[[1, -1, 0, -1, -1, -1, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0]]
[[1, -1, 0, -1, -1, -1, 0, -1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, 0]]


In [ ]:
#type(page_rank.ranki)

In [ ]:
type(checko)
test = ny.asarray(checko)
type(test)
#len(test)

numpy.ndarray

In [ ]:
import joblib
import pickle

#with open("model.pkl","rb") as f:
#    model = pickle.load(f)=[]
#with open('model.plk', 'rb') as infile:
#    model = pickle.load(infile)
#print(type(classifier))
classifier = joblib.load("Model.sav")
#checkprediction = main(url)
predictions = classifier.predict(checko)

# prediction = classifier.main(url)
print(predictions)

[0.112]


In [ ]:
type(classifier)

sklearn.ensemble.forest.RandomForestRegressor